# EEG - Flow

## 3. Manually select bad channels

The bad channels and the bad segments must be annotated and excluded from subsequent analysis. 

In [1]:
from itertools import chain

from mne import read_annotations
from mne.io import read_raw_fif, write_info, read_info
from mne.preprocessing import compute_bridged_electrodes, interpolate_bridged_electrodes
from pyprep import NoisyChannels

from eeg_flow.config import load_config
from eeg_flow.utils.annotations import merge_bad_annotations
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files
from eeg_flow.viz import plot_bridged_electrodes


_, derivatives_folder, experimenter = load_config()

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [2]:
participant = 3  # int
group =  1  # int
task = "oddball"  # str
run = 1  # int

derivatives_folder_preprocessed_p = get_folder(derivatives_folder / "preprocessed", participant, group)
fname_stem = get_fname(participant, group, task, run)

# create locks
derivatives = (
    derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step3_"+experimenter+"_info.fif"),
    derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step3_"+experimenter+"_oddball_with_bads_annot.fif"),
)
locks = lock_files(*derivatives)

# load raw recording
raw = read_raw_fif(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step1_raw.fif"), preload=True)
annot = read_annotations(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_oddball_with_bads_annot.fif"))
info = read_info(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_info.fif"))

#is this necessary here?
raw.annotations.__add__(annot)

##merge infos? if necessary?

Opening raw data file L:\EEG_Flow_data\derivatives\preprocessed\sub-P03-G1\sub-P03-G1_task-oddball_run-1\sub-P03-G1_task-oddball_run-1_step1_raw.fif...
Isotrak not found
    Range : 6216 ... 384492 =      6.070 ...   375.480 secs
Ready.
Reading 0 ... 378276  =      0.000 ...   369.410 secs...


<Annotations | 716 segments: bad_standard (6), bad_target (1), novel (72), ...>

## 2.1 Visual inspection and annotations of bad channels


In [4]:
raw.filter(
    l_freq=1.0,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-12 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-12 dB cutoff frequency: 45.00 Hz)
- Filter length: 3381 samples (3.302 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


<Raw | sub-P03-G1_task-oddball_run-1_step1_raw.fif, 68 x 378277 (369.4 s), ~196.3 MB, data loaded>

In [5]:
raw.plot(theme="light")

2023-04-06 20:04:15,643 - qdarkstyle - INFO - QSS file successfully loaded.
2023-04-06 20:04:15,644 - qdarkstyle - INFO - Found version patches to be applied.
2023-04-06 20:04:15,644 - qdarkstyle - INFO - Found application patches to be applied.


Channels marked as bad:
none


## 2.4 Save derivatives

The updated annotations can now be saved alongside the selected bad channels.

In [ ]:
#what should be saved here?

In [ ]:
fname = derivatives_folder / (fname_stem + "_step3_"+experimenter+"_info.fif")
assert not fname.exists()  # write_info always overwrites 
write_info(fname, raw.info)
derivatives_folder / (fname_stem + "_step3_"+experimenter+"_oddball_with_bads_annot.fif"),
annotations.save(fname, overwrite=False)

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway